In [1]:
import pandas as pd
import numpy as np
import re
from copy import deepcopy

In [2]:
rawdata = pd.read_excel('./rawdata.xlsx') # 720 -> 628 -> 590 -> 567
rawdata = rawdata[['gene','Promoter_Seq','RBS_Seq','Gene_180bp','平均flu/OD']]
rawdata['平均flu/OD'] = pd.to_numeric(rawdata['平均flu/OD'], errors='coerce')
rawdata['平均flu/OD'] = rawdata['平均flu/OD'].astype(float) + 3000
rawdata = rawdata.dropna()
print(len(rawdata))
rawdata = rawdata[rawdata['Promoter_Seq'].apply(lambda x: 20 < len(x) < 100)] # 683 -> 580
print(len(rawdata))
rawdata = rawdata[rawdata['RBS_Seq'].apply(lambda x: 10 < len(x) < 50)] # 683 -> 580
print(len(rawdata))
rawdata = rawdata[~rawdata.gene.str.contains("_WT")] 
print(len(rawdata))

# P 20 <100 R 10 < 50 CDS ==180
rawdata.index = range(len(rawdata))
rawdata # 720 总数 -> 628 去空-> 590 去WT -> 567 去处异常长度

628
567
567
567


,gene,Promoter_Seq,RBS_Seq,Gene_180bp,平均flu/OD
0,atoB(thl)_100,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG,ATCTTAATCTAGCGTGGGAGAGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,119164.059432
1,atoB(thl)_130,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,TCACACAGGAAAGGCCCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,164210.862842
2,atoB(thl)_140,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCTGGC,ATCTTAATCTAGCTAGGGATATTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,14266.357622
3,atoB(thl)_150,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,ATCTTAATCTAGCTAGGGAGTGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,104460.286090
4,atoB(thl)_160,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT,ATCTTAATCTAGCTAGGGAACGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,20202.330833
...,...,...,...,...,...
562,GFP_320,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAGTG,AAAGATCTTTTAAGAAGGAGATATACATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,24476.305921
563,GFP_40,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCCGAC,ATCTTAATCTAGCGGGGGATTTTTTCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,8748.363110
564,GFP_50,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA,TCTAGAGAAAGATTGGACGTACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,5553.358256
565,GFP_70,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,TCTAGAGAAAGAGTGGAAAAACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,8146.421157


### machine learning

In [3]:
mldata = deepcopy(rawdata)
mldata['Seq'] = mldata['Promoter_Seq']+mldata['RBS_Seq']+mldata['Gene_180bp']
mldata['Seq'] = mldata['Seq'].apply(lambda x: x.upper())
# # 使用apply和lambda函数筛选出长度大于等于250的行
# mldata = mldata[mldata['Seq'].apply(lambda x: 250 < len(x) < 300)] # 683 -> 580
mldata['Seq'] = mldata['Seq'].apply(lambda x: x[:248])
mldata['平均flu/OD'] = mldata['平均flu/OD'].apply(np.log1p)
# 使用正则表达式删除非"ATCG"的字符
mldata['Seq'] = mldata['Seq'].str.replace(r'[^ATCG]', '')
mldata = mldata.dropna()
mldata

,gene,Promoter_Seq,RBS_Seq,Gene_180bp,平均flu/OD,Seq
0,atoB(thl)_100,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG,ATCTTAATCTAGCGTGGGAGAGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,11.688265,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG...
1,atoB(thl)_130,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,TCACACAGGAAAGGCCCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,12.008913,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...
2,atoB(thl)_140,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCTGGC,ATCTTAATCTAGCTAGGGATATTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,9.565730,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCT...
3,atoB(thl)_150,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,ATCTTAATCTAGCTAGGGAGTGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,11.556572,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...
4,atoB(thl)_160,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT,ATCTTAATCTAGCTAGGGAACGTTTCATA,ATGAAAGAAGTTGTAATAGCTAGTGCAGTAAGAACAGCGATTGGAT...,9.913603,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT...
...,...,...,...,...,...,...
562,GFP_320,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAGTG,AAAGATCTTTTAAGAAGGAGATATACATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,10.105502,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAG...
563,GFP_40,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCCGAC,ATCTTAATCTAGCGGGGGATTTTTTCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,9.076736,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCC...
564,GFP_50,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA,TCTAGAGAAAGATTGGACGTACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,8.622338,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA...
565,GFP_70,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,TCTAGAGAAAGAGTGGAAAAACCATA,atgggatccatgtcgaagggcgaggagctgttcaccggcgtcgtcc...,9.005457,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...


In [4]:
mldata.to_csv("ML.csv",index=False)

##### BioAutoMATED

In [5]:
atdata = deepcopy(rawdata)
atdata['Seq'] = atdata['Promoter_Seq']+atdata['RBS_Seq']+atdata['Gene_180bp']
atdata['Seq'] = atdata['Seq'].apply(lambda x: x.upper())
# 使用apply和lambda函数筛选出长度大于等于250的行
# atdata = atdata[atdata['Seq'].apply(lambda x: 250 < len(x) < 300)] # 683 -> 580
# atdata['Seq'] = atdata['Seq'].apply(lambda x: x[:250])
atdata['平均flu/OD'] = atdata['平均flu/OD'].apply(np.log1p)
# 使用正则表达式删除非"ATCG"的字符
atdata['Seq'] = atdata['Seq'].str.replace(r'[^ATCG]', '')
atdata = atdata.dropna()

atdata = atdata[['gene','Seq','平均flu/OD']]
atdata.columns = ['Gene','Seq','OD']
atdata.to_csv("Exp.csv",index=False)
atdata

,Gene,Seq,OD
0,atoB(thl)_100,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGCCGTCG...,11.688265
1,atoB(thl)_130,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCCATAATT...,12.008913
2,atoB(thl)_140,CGCGCCTTGACATAAAGTCTAACCTATAGGTATAATGTGTGGATCT...,9.565730
3,atoB(thl)_150,CGCGCCTTGACAATTAATCATCCGGCTCGTATAATGTGTGGAATTG...,11.556572
4,atoB(thl)_160,CGCGCCTTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGCTTAAT...,9.913603
...,...,...,...
562,GFP_320,CGCGCCTTGACATTTATCCCTTGCGGCGATATAATGTGTGGATAAG...,10.105502
563,GFP_40,CGCGCCTTGACATAAAGTCTAACCTATAGGCATAATTATTTCATCC...,9.076736
564,GFP_50,CGCGCCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCACGAA...,8.622338
565,GFP_70,CGCGCCAAAAAGAGTATTGACTTCGCATCTTTTTGTACCTATAATA...,9.005457
